In [20]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from drone_model import DroneSimulator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set the trajectory


In [21]:
# Time
dt = 0.1
fs = 1 / dt
tsim = np.arange(0, 5.0 + dt/2, step=dt)

In [22]:
# Ambient wind speed & direction
w = 2.0 * np.ones_like(tsim)  # ambient wind speed magnitude
zeta = np.pi * np.ones_like(tsim)  # ambient wind direction

# Altitude
z = 2.0 * np.ones_like(tsim)  # altitude

# Velocity magnitude and direction
g = 1.0 * np.ones_like(tsim)  # ground speed magnitude
psi_global = (np.pi/4) * np.ones_like(tsim)  # global course direction
psi_global[30:] = psi_global[30] + np.pi/2

# Global frame x-y velocities
v_x_global = g * np.cos(psi_global)
v_y_global = g * np.sin(psi_global)

# Heading
psi = (np.pi/4) * np.ones_like(tsim)
psi[30:] = psi[30] + np.pi/2

# Body-level frame x-y velocities
v_x = v_x_global * np.cos(psi) + v_y_global * np.sin(psi)
v_y = -v_x_global * np.sin(psi) + v_y_global * np.cos(psi)

# Simulate trajectory


In [23]:
mpc_horizon = 10
mpc_control_penalty = 1e-2

In [25]:
# Create simulator
simulator = DroneSimulator(dt=dt, mpc_horizon=mpc_horizon, r_u=mpc_control_penalty)

# Change parameters
print(simulator.params.__dict__.keys())
simulator.params.M = 2.0  # example change the mass

dict_keys(['g', 'm', 'M', 'Mm', 'L', 'R', 'I_x', 'I_y', 'I_z', 'b', 'd', 'C'])


In [26]:
# Update the setpoints
simulator.update_setpoint(v_x=v_x, v_y=v_y, psi=psi, z=z, w=w, zeta=zeta)

# Run
t_sim, x_sim, u_sim, y_sim = simulator.simulate(x0=None, mpc=True, return_full_output=True)
sim_data = pd.DataFrame(y_sim)
sim_data.insert(0, 'time', t_sim)
print('Done.')

Done.


In [27]:
sim_data

,time,x,y,z,v_x,v_y,v_z,phi,theta,psi,...,r_x,r_y,a,gamma,v_x_dot,v_y_dot,q,alpha,u_x,u_y
0,0.0,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.785398,...,0.500000,0.000000,2.797933,-0.529903,-0.009285,0.005439,0.010761,2.611690,0.000000,-0.000000
1,0.1,0.070659,0.070697,2.045255,0.999091,0.000386,0.905036,0.000291,0.000748,0.785549,...,0.488492,0.000189,2.796874,-0.530056,-0.008057,0.001948,0.008289,2.904344,0.031792,-0.012370
2,0.2,0.141225,0.141372,2.176608,0.998382,0.000543,1.721976,0.000703,0.001778,0.785850,...,0.458687,0.000250,2.796030,-0.530345,-0.004604,0.000602,0.004643,3.011466,0.121455,-0.048017
3,0.3,0.211723,0.212036,2.384679,0.997954,0.000650,2.439395,0.000758,0.001858,0.786088,...,0.418486,0.000272,2.795487,-0.530555,-0.002351,0.000849,0.002499,2.795071,0.179974,-0.073468
4,0.4,0.282175,0.282696,2.658989,0.997648,0.000789,3.046774,0.000636,0.001488,0.786234,...,0.375198,0.000297,2.795079,-0.530668,-0.001954,0.001173,0.002279,2.600972,0.190248,-0.081307
5,0.5,0.352587,0.353355,2.988073,0.997388,0.000933,3.534882,0.000560,0.001261,0.786343,...,0.333790,0.000312,2.794727,-0.530745,-0.001472,0.000939,0.001746,2.573863,0.202718,-0.090088
6,0.6,0.422967,0.424015,3.359624,0.997205,0.001043,3.896112,0.000525,0.001148,0.786442,...,0.296820,0.000310,2.794463,-0.530813,-0.000651,0.000558,0.000857,2.432814,0.224052,-0.102455
7,0.7,0.493324,0.494677,3.760670,0.997100,0.001122,4.124794,0.000491,0.001042,0.786526,...,0.265139,0.000298,2.794291,-0.530866,-0.000028,0.000350,0.000351,1.650867,0.240220,-0.113204
8,0.8,0.563665,0.565345,4.177783,0.997054,0.001182,4.217452,0.000461,0.000948,0.786589,...,0.238656,0.000283,2.794189,-0.530899,0.000441,0.000209,0.000488,0.443562,0.252397,-0.122708
9,0.9,0.633997,0.636018,4.597306,0.997059,0.001225,4.173023,0.000438,0.000874,0.786630,...,0.216879,0.000266,2.794150,-0.530913,0.000863,0.000050,0.000865,0.057637,0.263392,-0.131940


In [9]:
simulator.plot_trajectory()